<a href="https://colab.research.google.com/github/VDai1999/ConcertTicketSale/blob/main/Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import packages
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data
import csv
import pandas as pd

In [ ]:
client_id = ""
client_secret = ""

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

In [ ]:
def song_features(result):
  '''
  Get songs' features
  '''
  
    songFeature = {}
    track = result['tracks']['items'][0]['uri']
    features = sp.audio_features(track)
    cols = ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 
            'liveness', 'valence', 'tempo', 'type', 'duration_ms']
    
    songFeature['artist'] = result['tracks']['items'][0]['artists'][0]['name']
    songFeature['song'] = result['tracks']['items'][0]['name']
    
    for col in cols:
        songFeature[col] = features[0][col]
        
    pop = sp.track(track)
    songFeature['popularity'] = pop['popularity']
    
    return songFeature

In [ ]:
df = pd.read_csv('Book3.csv')

In [ ]:
df.head()

In [ ]:
df['Song'] = df.Song.str.replace(r"\(.*\)","").str.replace(r"\[.*\]","")

In [ ]:
def main():
  
  stop = False
  listSongFeature = []
  
  
  for i in range (len(df)):
      artist = df['Artist'][i]
      song = df['Song'][i].rstrip()
      result = sp.search(q=artist+ " " + song, limit=1)
      sf = song_features(result) 
      listSongFeature.append(sf)
      
  csv_columns = ['artist', 'song', 'danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 
                'liveness', 'valence', 'tempo', 'type', 'duration_ms', 'popularity']
  csv_file = "features.csv"
  
  try:
      with open(csv_file, 'w') as csvfile:
          writer = csv.writer(csvfile)
          writer.writerow(csv_columns)
          
          for lsf in listSongFeature:
              temp = list()
              for key in lsf.keys():
                  temp.append(lsf[key])
              writer.writerow(temp)
          
  except IOError:
      print("I/O error")
     

In [ ]:
if __name__ == '__main__':
    main()